In [1]:
import json 
# dataset load
def load_sales_data(fileName):
    with open(fileName, "r") as f:
        data = json.load(f)
    return data

sales_data = load_sales_data("sales_data.json")

In [3]:

# Data Cleaning 

#✅ 1. Remove Duplicate Transactions

def remove_duplicates(data):
    seen = set()   # Keeps track of order_ids we've already seen
    cleaned = []    # This will store the final cleaned data
    for row in data:  # Go through each row in the dataset
        oid = row.get("order_id")  # Try to get the 'order_id' from the row
        if oid and oid not in seen:   # If it's not already seen
            cleaned.append(row)   # Keep the row
            seen.add(oid)   # Mark this order_id as seen
    return cleaned          # return [row for row in data if (row.get("order_id") and row.get("order_id") not in seen and not seen.add(row.get("order_id")))] this is called List Comprehension

sales_data = remove_duplicates(sales_data)


# ✅ 2. Fill missing 'region' values with "Unknown"
def fill_missing_regions(data):
    for row in data:
        region = row.get("region")  # Get region (could be None, "", or missing)
        if not region:              # If empty string, None, or missing
            row["region"] = "Unknown"
    return data

sales_data = fill_missing_regions(sales_data)


# 🔡 Clean Product Names

def titeled_product(data):
    for row in data:
        product = row.get("product")
        if product:
            row["product"]= product.strip().title()
        else:
            row["product"] = "Unknown"
    return data


sales_data = titeled_product(sales_data)


# 💰 Convert "amount" or "price" to float

def convert_amount_to_float(data):
    for row in data:
        amount = row.get("unit_price")
        try:
            row["unit_price"] = float(amount)
        except (ValueError, TypeError):
            row["unit_price"] = 0.0
    return data

sales_data = convert_amount_to_float(sales_data)

# 📅Date Standardizer

from datetime import datetime

def standardize_date_format(date_str):
    if not date_str or not isinstance(date_str, str):
        return None  # skip bad input
    
    # Try these common formats
    formats = [
        "%d/%m/%Y",     # 13/06/2024
        "%m/%d/%Y",     # 06/13/2024
        "%Y/%m/%d",     # 2024/06/13
        "%Y-%m-%d",     # 2024-06-13
        "%b %d, %Y",    # Jun 13, 2024
        "%d-%b-%Y",     # 13-Jun-2024
    ]

    for fmt in formats:
        try:
            dt = datetime.strptime(date_str.strip(), fmt)
            return dt.strftime("%Y-%m-%d")  # output format
        except ValueError:
            continue

    return None  # If all formats fail

def standardize_dates(data):
    for row in data:
        original = row.get("date", "")
        standardized = standardize_date_format(original)
        if standardized:
            row["date"] = standardized
        else:
            row["date"] = None  # optional: or delete the row entirely
    return data

sales_data = standardize_dates(sales_data)

with open("cleaned_sales_data.json", "w") as f:   #this return a new file without duplicate order_id 
    json.dump(sales_data, f, indent=4)

In [5]:
# 🧠 Exploratory Data Analysis (EDA) 

# Basic Stats

# 1. 🔢 Total Number of Transactions

def total_transactions(data):
    return len(data)

# 2. 💰 Total Revenue (Sum of all amounts)

def total_revenue(data):
    total = 0
    for row in data:
        amount = row.get("unit_price", 0)
        total += amount
    return total

# 3. 📦 Count and name of Unique Products

def unique_products(data):
    product_set = set()
    for row in data:
        product = row.get("product", "unknown")
        product_set.add(product)
    return len(product_set), sorted(product_set)

# 4. 🌍 Count and name  of Unique Regions

def unique_regions(data):
    region_set = set()
    for row in data:
        region = row.get("region", "unknown")
        region_set.add(region)
    return len(region_set), sorted(region_set)

# Printing the report

total_txns = total_transactions(sales_data)
total_rev = total_revenue(sales_data)
prod_count, prod_names = unique_products(sales_data)
region_count, region_names = unique_regions(sales_data)

print("📊 LEVEL 1 – BASIC STATS")
print(f"Total Transactions     : {total_txns}")
print(f"Total Revenue (₹)      : {total_rev:,.2f}")
print(f"Unique Products        : {prod_count} → {prod_names}")
print(f"Unique Regions         : {region_count} → {region_names}")

📊 LEVEL 1 – BASIC STATS
Total Transactions     : 41
Total Revenue (₹)      : 322,625.00
Unique Products        : 11 → ['Backpack', 'Book', 'Headphones', 'Laptop', 'Notebook', 'Pen', 'Shoes', 'Smart-Watch', 'Smartphone', 'T-Shirt', 'Tshirt']
Unique Regions         : 5 → ['East', 'North', 'South', 'Unknown', 'West']


In [5]:
# 🧠 Exploratory Data Analysis (EDA) Level - 2

# ✅ Grouped Summaries (Level 2)


# Sales by Product 

def revenue_by_product(data):
    product_revenue = {}  # key = product, value = total amount

    for row in data:
        product = row.get("product", "").strip().title()
        amount = row.get("unit_price", 0)

        # Skip 'Unknown' and invalid amounts
        if product.lower() != "unknown" and isinstance(amount, (int, float)):
            if product not in product_revenue:
                product_revenue[product] = 0
            product_revenue[product] += amount

    return product_revenue

product_totals = revenue_by_product(sales_data)

# 📍 Display product-wise Revenue
print("\n🧺 Revenue by Product:")
for product, total in sorted(product_totals.items(), key=lambda x: -x[1]):
    print(f"{product:15} → ₹ {total:,.2f}")





# Sales by Region 🚀
def revenue_by_region(data):
    region_revenue = {}  # key = region, value = total revenue

    for row in data:
        region = row.get("region", "Unknown").strip().title()
        amount = row.get("unit_price", 0)

        if region and isinstance(amount, (int, float)):
            region_revenue[region] = region_revenue.get(region, 0) + amount

    return region_revenue

# 📍 Display Region-wise Revenue
region_totals = revenue_by_region(sales_data)

print("\n📍 Revenue by Region:")
for region, total in region_totals.items():
    print(f"{region:10} → ₹ {total:,.2f}")



# Monthly Sales 📅

def revenue_by_month(data):
    monthly_totals = {}

    for row in data:
        date = row.get("date")
        amount = row.get("unit_price", 0)

        # Only use rows with valid date and numeric amount
        if date and isinstance(amount, (int, float)):
            month = date[:7]  # Get "YYYY-MM"
            if month not in monthly_totals:
                monthly_totals[month] = 0
            monthly_totals[month] += amount

    return monthly_totals

# 📍 Display Monthly Summary

month_summary = revenue_by_month(sales_data)

print("\n📅 Monthly Revenue Summary:")
for month, total in sorted(month_summary.items()):
    print(f"{month} → ₹ {total:,.2f}")





🧺 Revenue by Product:
Laptop          → ₹ 250,000.00
Smartphone      → ₹ 36,000.00
Backpack        → ₹ 10,500.00
Headphones      → ₹ 8,000.00
Shoes           → ₹ 7,500.00
Smart-Watch     → ₹ 5,000.00
Tshirt          → ₹ 2,400.00
T-Shirt         → ₹ 1,600.00
Book            → ₹ 1,200.00
Notebook        → ₹ 300.00
Pen             → ₹ 125.00

📍 Revenue by Region:
Unknown    → ₹ 132,920.00
South      → ₹ 25,100.00
East       → ₹ 105,485.00
North      → ₹ 8,720.00
West       → ₹ 50,400.00

📅 Monthly Revenue Summary:
2023-01 → ₹ 55,525.00
2023-02 → ₹ 6,625.00
2023-03 → ₹ 106,860.00
2023-04 → ₹ 50,860.00
2023-05 → ₹ 12,545.00
2023-06 → ₹ 3,000.00
2023-07 → ₹ 12,000.00
2023-09 → ₹ 25.00
2023-10 → ₹ 5,000.00


In [6]:
# 🧠 Exploratory Data Analysis (EDA) Level - 3

# ✅ Deep Insights (Level 3)


# Top 5 products & bottom 5 🏆🔻

def top_bottom_products(data, top_n=5):
    product_totals = revenue_by_product(data)

    sorted_products = sorted(product_totals.items(), key=lambda x: x[1], reverse=True)
    
    top_5 = sorted_products[:top_n]
    bottom_5 = sorted_products[-top_n:]

    return top_5, bottom_5

# 🖨️ Print Top & Bottom

top_5, bottom_5 = top_bottom_products(sales_data)

print("\n🏆 Top 5 Products by Revenue:")
for product, total in top_5:
    print(f"{product:15} → ₹ {total:,.2f}")

print("\n🔻 Bottom 5 Products by Revenue:")
for product, total in bottom_5:
    print(f"{product:15} → ₹ {total:,.2f}")

 

# Best/worst performing region 📍

def best_worst_region(data):
    region_totals = revenue_by_region(data)

    sorted_regions = sorted(region_totals.items(), key=lambda x: x[1], reverse=True)

    best = sorted_regions[0]
    worst = sorted_regions[-1]

    return best, worst

# 🖨️ Print Region Insight

best, worst = best_worst_region(sales_data)

print(f"\n📍 Best Performing Region: {best[0]} → ₹ {best[1]:,.2f}")
print(f"📍 Worst Performing Region: {worst[0]} → ₹ {worst[1]:,.2f}")




# Trends over time (growth/decline) 📈

def monthly_trend(data):
    month_totals = revenue_by_month(data)
    trend = sorted(month_totals.items())  # sort by month

    print("\n📈 Monthly Sales Trend:")
    for i, (month, total) in enumerate(trend):
        print(f"{month} → ₹ {total:,.2f}")
        if i > 0:
            diff = total - trend[i-1][1]
            direction = "⬆️" if diff > 0 else "⬇️"
            print(f"   Change from last month: {direction} ₹ {abs(diff):,.2f}")

monthly_trend(sales_data)



🏆 Top 5 Products by Revenue:
Laptop          → ₹ 250,000.00
Smartphone      → ₹ 36,000.00
Backpack        → ₹ 10,500.00
Headphones      → ₹ 8,000.00
Shoes           → ₹ 7,500.00

🔻 Bottom 5 Products by Revenue:
Tshirt          → ₹ 2,400.00
T-Shirt         → ₹ 1,600.00
Book            → ₹ 1,200.00
Notebook        → ₹ 300.00
Pen             → ₹ 125.00

📍 Best Performing Region: Unknown → ₹ 132,920.00
📍 Worst Performing Region: North → ₹ 8,720.00

📈 Monthly Sales Trend:
2023-01 → ₹ 55,525.00
2023-02 → ₹ 6,625.00
   Change from last month: ⬇️ ₹ 48,900.00
2023-03 → ₹ 106,860.00
   Change from last month: ⬆️ ₹ 100,235.00
2023-04 → ₹ 50,860.00
   Change from last month: ⬇️ ₹ 56,000.00
2023-05 → ₹ 12,545.00
   Change from last month: ⬇️ ₹ 38,315.00
2023-06 → ₹ 3,000.00
   Change from last month: ⬇️ ₹ 9,545.00
2023-07 → ₹ 12,000.00
   Change from last month: ⬆️ ₹ 9,000.00
2023-09 → ₹ 25.00
   Change from last month: ⬇️ ₹ 11,975.00
2023-10 → ₹ 5,000.00
   Change from last month: ⬆️ ₹ 4,975.00


In [7]:
# 📊 Full EDA Report (All Levels)

# ✅ 1. Export Level 1 Summary

import csv
def export_level1_summary(data, file_name="level1_summary.csv"):
    total_revenue = sum(
    row.get("unit_price", 0)
    for row in data
    if isinstance(row.get("unit_price", 0), (int, float))
)
    total_transactions = len(data)

    unique_products = set()
    unique_regions = set()
    for row in data:
        product = row.get("product")
        region = row.get("region")
        if product and product != "unknown":
            unique_products.add(product)
        if region and region != "unknown":
            unique_regions.add(region)

    with open(file_name, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Metric", "Value"])
        writer.writerow(["Total Revenue", total_revenue])
        writer.writerow(["Total Transactions", total_transactions])
        writer.writerow(["Unique Products", len(unique_products)])
        writer.writerow(["Unique Regions", len(unique_regions)])


# ✅ 2. Export Product Summary

def export_product_summary(data, file_name="product_summary.csv"):
    product_totals = revenue_by_product(data)

    with open(file_name, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Product", "Revenue"])

        for product, revenue in sorted(product_totals.items(), key=lambda x: x[1], reverse=True):
            writer.writerow([product, revenue])

# ✅ 3. Export Region Summary

def export_region_summary(data, file_name="region_summary.csv"):
    region_totals = revenue_by_region(data)

    with open(file_name, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Region", "Revenue"])

        for region, revenue in sorted(region_totals.items(), key=lambda x: x[1], reverse=True):
            writer.writerow([region, revenue])


# ✅ 4. Export Monthly Trend

def export_monthly_trend(data, file_name="monthly_trend.csv"):
    month_totals = revenue_by_month(data)
    trend = sorted(month_totals.items())

    with open(file_name, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Month", "Revenue", "Change From Last Month"])

        prev = None
        for month, total in trend:
            if prev is None:
                writer.writerow([month, total, ""])
            else:
                diff = total - prev
                writer.writerow([month, total, diff])
            prev = total


# ✅ 5. Export Top/Bottom 5 Products

def export_top_bottom_products(data, file_name="top_bottom_products.csv"):
    top_5, bottom_5 = top_bottom_products(data)

    with open(file_name, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Type", "Product", "Revenue"])

        for product, revenue in top_5:
            writer.writerow(["Top 5", product, revenue])
        for product, revenue in bottom_5:
            writer.writerow(["Bottom 5", product, revenue])


# ▶️ Final Step: Run All Exports

export_level1_summary(sales_data)
export_product_summary(sales_data)
export_region_summary(sales_data)
export_monthly_trend(sales_data)
export_top_bottom_products(sales_data)

